# Big Data
# BD02 Dataframes



## <font color='blue'>**Dataframes**</font>

Un Spark DataFrame es una colección inmutable de datos distribuidos dentro de un clúster. Los datos dentro de un DataFrame se organizan en columnas con nombre que se pueden comparar con tablas en una base de datos relacional. Los DataFrames aprovechan los avances en el proyecto tungsteno
 y Catalyst Optimizer. Estas dos mejoras aportan la
rendimiento de PySpark a la par con el de Scala o Java.

Catalyst Optimizer se encuentra en el core de Spark SQL y potencia tanto las consultas SQL ejecutadas en los datos como en los DataFrames. El proceso comienza con la consulta que se envía al motor. Primero se optimiza el plan lógico de ejecución. Basado en el plan lógico optimizado, se derivan múltiples planes físicos y se envían a través de un optimizador de costos. Luego, el plan seleccionado y más rentable se traduce (utilizando optimizaciones de generación de código implementadas como parte del proyecto de tungsteno) en un código de ejecución optimizado basado en RDD.

Aunque Python no es un lenguaje fuertemente tipado, los DataFrames en PySpark sí lo son. A diferencia de los RDD, cada elemento de una columna DataFrame tiene un tipo específico (todos se enumeran en el submódulo pyspark.sql.types) y todos los datos deben ajustarse al esquema especificado. El Catalyst Optimizer se encuentra en el core de Spark SQL y potencia tanto las consultas SQL ejecutadas contra los datos como DataFrames. El proceso comienza con la consulta que se envía al motor. Primero se optimiza el plan lógico de ejecución. Basado en el plan lógico optimizado, se derivan múltiples planes físicos y se envían a través de un optimizador de costos. Luego, el plan seleccionado y más rentable se traduce (utilizando optimizaciones de generación de código implementadas como parte del proyecto de tungsteno) en un código de ejecución optimizado basado en RDD.

Cambiar a usar DataFrames no significa que debamos
abandonar los RDD. DataFrames todavía usa RDD, pero de objetos Row(...),  En este notebook, aprenderemos cómo interactuar con  un DataFrame.

In [ ]:
# Esto solo lo utilizamos para instalar las librerias 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz       
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Creemos algunos ejemplos. 

In [ ]:
sample_data = spark.sparkContext.parallelize([
      (1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)
    , (2, 'MacBook', 2016, '12"', '8GB', '256GB SSD', 11.04, 7.74, 0.52, 2.03)
    , (3, 'MacBook Air', 2016, '13.3"', '8GB', '128GB SSD', 12.8, 8.94, 0.68, 2.96)
    , (4, 'iMac', 2017, '27"', '64GB', '1TB SSD', 25.6, 8.0, 20.3, 20.8)
])

In [ ]:
sample_data.take(2)

[(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02),
 (2, 'MacBook', 2016, '12"', '8GB', '256GB SSD', 11.04, 7.74, 0.52, 2.03)]

### Creating DataFrames

In [ ]:
sample_data_df = spark.createDataFrame(
    sample_data
    , [
        'Id'
        , 'Model'
        , 'Year'
        , 'ScreenSize'
        , 'RAM'
        , 'HDD'
        , 'W'
        , 'D'
        , 'H'
        , 'Weight'
    ]
)

In [ ]:
sample_data.take(1)

[(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)]

In [ ]:
sample_data_df.take(1)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02)]

In [ ]:
sample_data_df.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
sample_data_df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)



### From JSON

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sample_data_json_df = (
    spark
    .read
    .json('/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/DataFrames_sample.json')
)

In [ ]:
sample_data_json_df.show()

+----+----+---------+-----------+----+-----------+-----+-------+-----+---+
|   D|   H|      HDD|      Model| RAM| ScreenSize|    W| Weight| Year| Id|
+----+----+---------+-----------+----+-----------+-----+-------+-----+---+
|9.48|0.61|512GB SSD|MacBook Pro|16GB|        15"|13.75|   4.02| 2015|  1|
|7.74|0.52|256GB SSD|    MacBook| 8GB|        12"|11.04|   2.03| 2016|  2|
|8.94|0.68|128GB SSD|MacBook Air| 8GB|      13.3"| 12.8|   2.96| 2016|  3|
| 8.0|20.3|  1TB SSD|       iMac|64GB|        27"| 25.6|   20.8| 2017|  4|
+----+----+---------+-----------+----+-----------+-----+-------+-----+---+



In [ ]:
sample_data_json_df.printSchema()

root
 |--  D: double (nullable = true)
 |--  H: double (nullable = true)
 |--  HDD: string (nullable = true)
 |--  Model: string (nullable = true)
 |--  RAM: string (nullable = true)
 |--  ScreenSize: string (nullable = true)
 |--  W: double (nullable = true)
 |--  Weight: double (nullable = true)
 |--  Year: long (nullable = true)
 |-- Id: long (nullable = true)



### From CSV

In [ ]:
sample_data_csv = (
    spark
    .read
    .csv(
        '/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/DataFrames_sample.csv'
        , header=True
        , inferSchema=True)
)

In [ ]:
sample_data_csv.show()

+---+-----------+-----+-----------+----+---------+-----+----+----+-------+
| Id|      Model| Year| ScreenSize| RAM|      HDD|    W|   D|   H| Weight|
+---+-----------+-----+-----------+----+---------+-----+----+----+-------+
|  1|MacBook Pro| 2015|        15"|16GB|512GB SSD|13.75|9.48|0.61|   4.02|
|  2|    MacBook| 2016|        12"| 8GB|256GB SSD|11.04|7.74|0.52|   2.03|
|  3|MacBook Air| 2016|      13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|   2.96|
|  4|       iMac| 2017|        27"|64GB|  1TB SSD| 25.6| 8.0|20.3|   20.8|
+---+-----------+-----+-----------+----+---------+-----+----+----+-------+



In [ ]:
sample_data_csv.printSchema()

root
 |-- Id: integer (nullable = true)
 |--  Model: string (nullable = true)
 |--  Year: integer (nullable = true)
 |--  ScreenSize: string (nullable = true)
 |--  RAM: string (nullable = true)
 |--  HDD: string (nullable = true)
 |--  W: double (nullable = true)
 |--  D: double (nullable = true)
 |--  H: double (nullable = true)
 |--  Weight: double (nullable = true)



### Acceder a RDD subyacentes

Cambiar a usar DataFrames no significa que debamos
abandonar los RDD. Bajo el capó, DataFrames todavía usa RDD, pero de objetos Row (...)

In [ ]:
sample_data_csv.rdd.take(1)

[Row(Id=1,  Model='MacBook Pro',  Year=2015,  ScreenSize='15"',  RAM='16GB',  HDD='512GB SSD',  W=13.75,  D=9.48,  H=0.61,  Weight=4.02)]

In [ ]:
sample_data.take(1)

[(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)]

In [ ]:
sample_data_json_df.rdd.take(1)

[Row( D=9.48,  H=0.61,  HDD='512GB SSD',  Model='MacBook Pro',  RAM='16GB',  ScreenSize='15"',  W=13.75,  Weight=4.02,  Year=2015, Id=1)]

## <font color='blue'>**Usando SQL para interactuar con dataframes**</font>

In [ ]:
import pyspark.sql.types as typ

sch = typ.StructType([
      typ.StructField('Id', typ.LongType(), False)
    , typ.StructField('Model', typ.StringType(), True)
    , typ.StructField('Year', typ.IntegerType(), True)
    , typ.StructField('ScreenSize', typ.StringType(), True)
    , typ.StructField('RAM', typ.StringType(), True)
    , typ.StructField('HDD', typ.StringType(), True)
    , typ.StructField('W', typ.DoubleType(), True)
    , typ.StructField('D', typ.DoubleType(), True)
    , typ.StructField('H', typ.DoubleType(), True)
    , typ.StructField('Weight', typ.DoubleType(), True)
])

In [ ]:
sample_data_rdd = spark.sparkContext.textFile('/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/DataFrames_sample.csv')

header = sample_data_rdd.first()

In [ ]:
sample_data_rdd.take(5)

['Id, Model, Year, ScreenSize, RAM, HDD, W, D, H, Weight',
 '1,MacBook Pro,2015,"15\\"",16GB,512GB SSD,13.75,9.48,0.61,4.02',
 '2,MacBook,2016,"12\\"",8GB,256GB SSD,11.04,7.74,0.52,2.03',
 '3,MacBook Air,2016,"13.3\\"",8GB,128GB SSD,12.8,8.94,0.68,2.96',
 '4,iMac,2017,"27\\"",64GB,1TB SSD,25.6,8.0,20.3,20.8']

In [ ]:
sample_data_rdd = (
    sample_data_rdd
    .filter(lambda row: row != header)
    .map(lambda row: row.split(','))
    .map(lambda row: (
                int(row[0])
                , row[1]
                , int(row[2])
                , row[3]
                , row[4]
                , row[5]
                , float(row[6])
                , float(row[7])
                , float(row[8])
                , float(row[9])
        )
    )
)

sample_data_schema = spark.createDataFrame(sample_data_rdd, schema=sch)
sample_data_schema.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
models_df = spark.sparkContext.parallelize([
      ('MacBook Pro', 'Laptop')
    , ('MacBook', 'Laptop')
    , ('MacBook Air', 'Laptop')
    , ('iMac', 'Desktop')
]).toDF(['Model', 'FormFactor'])

models_df.createOrReplaceTempView('models')

In [ ]:
sample_data_schema.createOrReplaceTempView('sample_data_view')

In [ ]:
spark.sql('''
    SELECT a.*
        , b.FormFactor
    FROM sample_data_view AS a
    LEFT JOIN models AS b
        ON a.Model == b.Model
    ORDER BY Weight DESC
''').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|   Desktop|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|    Laptop|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|    Laptop|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|    Laptop|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+



In [ ]:
spark.sql('''
    SELECT b.FormFactor
        , COUNT(*) AS ComputerCnt
    FROM sample_data_view AS a
    LEFT JOIN models AS b
        ON a.Model == b.Model
    GROUP BY FormFactor
''').show()

+----------+-----------+
|FormFactor|ComputerCnt|
+----------+-----------+
|    Laptop|          3|
|   Desktop|          1|
+----------+-----------+



## <font color='green'>**Ejercicio 1**</font>

En este ejercicio volvemos a trabajar con el archivo people.json. 

1. Leemos el archivo como un json
2. Filtramos los registros mayores de 21 años.
3. Agrupamos por edad y contamos la cantidad.

Realicemos algunas consultas SQL

4. Creamos una vista temporal con df.createOrReplaceTempView("people")
5. Realice SELECT * FROM people
6. Realice el select para recuperar las personas mayores de 13 y menores de 25
7. Escriba el dataframe leido desde json en formato parquet
8. Lea el archivo almacenado en parquet y dejelo en un dataframe.


In [ ]:
people_json = (
    spark
    .read
    .json('/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/people.json')
)

In [ ]:
people_json.show()

+----+--------+
| age|    name|
+----+--------+
|null| Michael|
|  30|    Andy|
|  19|  Justin|
|  11|     Ana|
|  44|Patricia|
|  89|     Leo|
+----+--------+



In [ ]:
people_json.take(1)

[Row(age=None, name='Michael')]

In [ ]:
people_json.createOrReplaceTempView("people")

In [ ]:
mt21 = spark.sql("SELECT * , COUNT(*) AS AgeCnt FROM people WHERE age > 21 GROUP BY age, name")

In [ ]:
mt21.show()

+---+--------+------+
|age|    name|AgeCnt|
+---+--------+------+
| 44|Patricia|     1|
| 89|     Leo|     1|
| 30|    Andy|     1|
+---+--------+------+



In [ ]:
mt21 = people_json.filter(people_json.age > 21).groupBy('age').count()

In [ ]:
mt21.show()

+---+-----+
|age|count|
+---+-----+
| 89|    1|
| 44|    1|
| 30|    1|
+---+-----+



In [ ]:
b1325 = spark.sql("SELECT * FROM people WHERE age BETWEEN 13 AND 25")

In [ ]:
b1325.show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+



In [ ]:
b1325.write.parquet("/tmp/output/people.parquet")

In [ ]:
parquet_df = spark.read.parquet("/tmp/output/people.parquet")

In [ ]:
parquet_df.show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+



<font color='green'>**Fin ejercicio 2**</font>

## <font color='blue'>**Transformaciones sobre dataframes**</font>

In [ ]:
sample_data_schema.cache().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
sample_data_schema.select('RAM').distinct().show()

+----+
| RAM|
+----+
|64GB|
|16GB|
| 8GB|
+----+



In [ ]:
sample_data_schema.dropDuplicates().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
sample_data_schema.dropna().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
sample_data_schema.fillna(0).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
sample_data_schema.filter(sample_data_schema.Year > 2015).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
sample_data_schema.freqItems(['RAM']).show()

+-----------------+
|    RAM_freqItems|
+-----------------+
|[64GB, 16GB, 8GB]|
+-----------------+



In [ ]:
sample_data_schema.orderBy('W').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
import pyspark.sql.functions as f

In [ ]:
sample_data_schema.orderBy(f.col('H').desc()).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
sample_data_schema_rep = (
    sample_data_schema
    .repartition(2, 'Year')
)

sample_data_schema_rep.rdd.getNumPartitions()

2

In [ ]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
sc=spark.sparkContext

In [ ]:
missing_df = sc.parallelize([
    (None, 36.3, 24.2)
    , (1.6, 32.1, 27.9)
    , (3.2, 38.7, 24.7)
    , (2.8, None, 23.9)
    , (3.9, 34.1, 27.9)
    , (9.2, None, None)
]).toDF(['A', 'B', 'C'])

missing_df.fillna(21.4).show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|21.4|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|21.4|23.9|
| 3.9|34.1|27.9|
| 9.2|21.4|21.4|
+----+----+----+



In [ ]:
miss_dict = (
    missing_df
    .agg(
        f.mean('A').alias('A')
        , f.mean('B').alias('B')
        , f.mean('C').alias('C')
    )
).toPandas().to_dict('records')[0]

missing_df.fillna(miss_dict).show()

+----+------------------+-----+
|   A|                 B|    C|
+----+------------------+-----+
|4.14|              36.3| 24.2|
| 1.6|              32.1| 27.9|
| 3.2|              38.7| 24.7|
| 2.8|35.300000000000004| 23.9|
| 3.9|              34.1| 27.9|
| 9.2|35.300000000000004|25.72|
+----+------------------+-----+



In [ ]:
missing_df.dropna().show()

+---+----+----+
|  A|   B|   C|
+---+----+----+
|1.6|32.1|27.9|
|3.2|38.7|24.7|
|3.9|34.1|27.9|
+---+----+----+



In [ ]:
missing_df.dropna(thresh=2).show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|null|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|null|23.9|
| 3.9|34.1|27.9|
+----+----+----+



In [ ]:
dupes_df = sc.parallelize([
      (1.6, 32.1, 27.9)
    , (3.2, 38.7, 24.7)
    , (3.9, 34.1, 27.9)
    , (3.2, 38.7, 24.7)
]).toDF(['A', 'B', 'C'])

dupes_df.dropDuplicates().show()

+---+----+----+
|  A|   B|   C|
+---+----+----+
|1.6|32.1|27.9|
|3.9|34.1|27.9|
|3.2|38.7|24.7|
+---+----+----+



In [ ]:
sample_data_schema.select('Model', 'ScreenSize').show()

+-----------+----------+
|      Model|ScreenSize|
+-----------+----------+
|MacBook Pro|    "15\""|
|    MacBook|    "12\""|
|MacBook Air|  "13.3\""|
|       iMac|    "27\""|
+-----------+----------+



In [ ]:
sample_data_schema.select('W').summary().show()
sample_data_schema.select('W').describe().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    25%|             11.04|
|    50%|              12.8|
|    75%|             13.75|
|    max|              25.6|
+-------+------------------+

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    max|              25.6|
+-------+------------------+



In [ ]:
another_macBookPro = sc.parallelize([
      (5, 'MacBook Pro', 2018, '15"', '16GB', '256GB SSD', 13.75, 9.48, 0.61, 4.02)
]).toDF(sample_data_schema.columns)

sample_data_schema.unionAll(another_macBookPro).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  5|MacBook Pro|2018|       15"|16GB|256GB SSD|13.75|9.48|0.61|  4.02|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [ ]:
(
    sample_data_schema
    .withColumn('HDDSplit', f.split(f.col('HDD'), ' '))
    .show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|    HDDSplit|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|[512GB, SSD]|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|  [1TB, SSD]|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+



In [ ]:
(
    sample_data_schema
    .groupBy('RAM')
    .count()
    .show()
)

+----+-----+
| RAM|count|
+----+-----+
|64GB|    1|
|16GB|    1|
| 8GB|    2|
+----+-----+



In [ ]:
(
    sample_data_schema
    .select(
        f.col('*')
        , f.split(f.col('HDD'), ' ').alias('HDD_Array')
    ).show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|   HDD_Array|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|[512GB, SSD]|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|  [1TB, SSD]|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+




## <font color='blue'>**Acciones sobre dataframes**</font>

In [ ]:
sample_data_schema.groupBy('Year').count().collect()

[Row(Year=2015, count=1), Row(Year=2016, count=2), Row(Year=2017, count=1)]

In [ ]:
sample_data_schema.select('W').describe().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    max|              25.6|
+-------+------------------+



In [ ]:
sample_data_schema.take(2)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='"15\\""', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02),
 Row(Id=2, Model='MacBook', Year=2016, ScreenSize='"12\\""', RAM='8GB', HDD='256GB SSD', W=11.04, D=7.74, H=0.52, Weight=2.03)]

In [ ]:
sample_data_schema.toPandas()

,Id,Model,Year,ScreenSize,RAM,HDD,W,D,H,Weight
0,1,MacBook Pro,2015,"""15\""""",16GB,512GB SSD,13.75,9.48,0.61,4.02
1,2,MacBook,2016,"""12\""""",8GB,256GB SSD,11.04,7.74,0.52,2.03
2,3,MacBook Air,2016,"""13.3\""""",8GB,128GB SSD,12.80,8.94,0.68,2.96
3,4,iMac,2017,"""27\""""",64GB,1TB SSD,25.60,8.00,20.30,20.80


In [ ]:
sample_data_schema.write.mode('overwrite').csv('sample_data_schema.csv')